In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']= '1'

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

In [3]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', 
                                  trust_remote_code = True, torch_dtype = torch.float16,
                                 use_flash_attention_2=True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
_ = model.cuda()

In [ ]:
import json
import os
from tqdm import tqdm

with open('news.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        filename = os.path.join('news', f'{no}.json')
        if os.path.exists(filename):
            continue
        l = json.loads(l)
        padded = tokenizer([l], return_tensors = 'pt', max_length = 20000).to('cuda')
        v = model.encode(padded).cpu().detach().numpy()[0].tolist()
        data = {
            'text': l,
            'v': v,
        }
        with open(filename, 'w') as f:
            json.dump(data, f)

1814901it [00:09, 231298.61it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2509215it [4:39:36, 40.56it/s]  